In [0]:
#importing libraries

import tensorflow as tf

import numpy as np


In [0]:
#for dynamic_decode we need => decoder , maximum_iteration (max_time in batch )

   #for decoder we need ==> cell , helper , intial_state , output layer
     
        # for helper we need ==> input_batch , sequence_length
       
              #let's go one by one


In [0]:
#helper function

input_data = np.random.randn(8,5,6).astype(np.float32)
max_sequence_length = 5
batch_size = 8

sequence_length = [2,4,5,3,2,5,7,8] #same size of batch

helper_function = tf.contrib.seq2seq.TrainingHelper(input_data,sequence_length,time_major=False)







In [0]:
#decoder 

cell=tf.contrib.rnn.LSTMCell(32)

initial_state = cell.zero_state(batch_size=batch_size,dtype=tf.float32)

vocab_size = 104

output_layer = tf.layers.Dense(vocab_size)

basic_decoder = tf.contrib.seq2seq.BasicDecoder(cell,helper_function,initial_state,output_layer)

In [24]:
#step decode

#we can manually decode step by step

#let's decode by one single step

initial_finished,initial_input,inital_state = basic_decoder.initialize()

step_output , step_state, next_input , step_finished=  basic_decoder.step(tf.constant(3),initial_input,initial_state)

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  rnn_output,lstm_tuple,next_input,step_finished = sess.run([step_output,step_state,next_input,step_finished])
  
  output , sample_id = rnn_output
  
  Memory , state_hidden_output = lstm_tuple
  
  print(output.shape)       #rnn_output                          #batch x vocab_size
  
  print(sample_id)         #argmax over rnn output               #batch_size
  
  print(Memory.shape)             #memory of state                #batch_size x lstm_hidden_size
  
  print(state_hidden_output.shape)    #hidden_output of state     #batch_size x lstm_hidden_size
  
  print(next_input.shape)        #next output for decoder state   #batch x dim
  
  print(step_finished)             #finished indication
  
  
  

(8, 104)
[ 50 103  48 102  48  81  99 102]
(8, 32)
(8, 32)
(8, 6)
[ True  True False  True  True False False False]


In [0]:
#now instead of doing this manually we want a loop , so it will unroll the rnn till all step finished , for that you can use
#your own loop , but tensorflow have function called dynamic_decode let's try that


dynam_decode = tf.contrib.seq2seq.dynamic_decode(basic_decoder,output_time_major=False,maximum_iterations=8)

In [29]:
#it will return 

#logits
#sample_id
# lstm_tuple
#final_finished

rnn_output , lstm_tuple , final_finished = dynam_decode


output , sample_id = rnn_output  
print(output.shape)   #batch x max_time x vocab
print(sample_id)      #batch size

last_state_memory , last_state_hidden_output = lstm_tuple

print(last_state_memory.shape)    #batch x lstm_hidden_size

print(last_state_hidden_output.shape)  #batch x lstm_hidden_size

print(final_finished)




(8, ?, 104)
Tensor("decoder_1/transpose_1:0", shape=(8, ?), dtype=int32)
(8, 32)
(8, 32)
Tensor("decoder_1/while/Exit_8:0", shape=(8,), dtype=int32)
